In [3]:
#This program loads the migrate message types into its related transaction detail table inside of the blockchain db
# This program accomplishes this in three steps
# 1. def connect_to_database: connects to blockchain database
#2. def get_most_recent_transaction_id(conn): pulls the related transaction_id from the header table
#3. def main: this function pull the data from their data paths in the json file and then pushes that data along with transaction_id to the detail table
# Libraries used: os, json, psycopg2, logging
import os
import json
import psycopg2  # Use psycopg2 for PostgreSQL
import logging

def connect_to_database():
    DB_NAME = 'blockchain'
    DB_USER = 'postgres'
    DB_PASSWORD = 'auth2020'
    DB_HOST = 'localhost'
    DB_PORT = '5432'

    try:
        # Create a PostgreSQL connection
        conn = psycopg2.connect(
            dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
        )
        cur = conn.cursor()
        print("Connection established.")  # Log successful connection (you can modify this)

        # Configure logging to write to a log file
        logging.basicConfig(filename='log.txt', level=logging.INFO)
        logging.info("Connection established with Database.")  # Log the same message to the file

        return conn  # Return the connection object

    except Exception as err:
        # Handle exceptions
        print("Unable to connect to the database.")  # Log error (you can modify this)
        print(err)  # Print the specific error message

        # Optionally, write the error message to a file (error.txt)
        with open('error.txt', 'a') as error_file:
            error_file.write(str(err) + '\n')

# Call the connect_to_database function
db_connection = connect_to_database()

Connection established.


In [4]:

def get_most_recent_transaction_id(conn):
    try:
        with conn.cursor() as cursor:
            # Query to retrieve the most recent transaction_id
            get_transaction_id_query = """
                SELECT transaction_id
                FROM transaction_header
                WHERE type = '/cosmwasm.wasm.v1.MsgMigrateContract'


                ORDER BY transaction_id DESC;
            """
            cursor.execute(get_transaction_id_query)
            result = cursor.fetchone()
            if result:
                return result[0]
            else:
                print("No transaction_id found.")
                return None
    except psycopg2.Error as e:
        print(f"Error retrieving transaction_id: {e}")
        return None

In [5]:
def main():
    block = '6041257'


    file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'

    try:
        if os.path.exists(file_path):
            print(f"Found JSON file: {file_path}")

            with open(file_path, 'r') as json_file:
                json_data = json.load(json_file)

                conn = connect_to_database()
                transaction_id = get_most_recent_transaction_id(conn)

                if transaction_id:
                    insert_queries = []
                    for message in json_data['tx']['body']['messages']: 
                        type = message["@type"]
                        contract = message["contract"]
                        sender =message["sender"]
                        code_id = message["code_id"]
                        #SQL Insert statement for FK and the data that was extracted from json file
                        msg_insert_query = f"""
                            INSERT INTO transaction_detail_msg_migrate(transaction_id,type, contract, sender, code_id)
                            VALUES ('{transaction_id}','{type}', '{contract}', '{sender}', '{code_id}');
                        """
                        insert_queries.append(msg_insert_query)

                    # Execute the insert queries
                    with conn.cursor() as cursor:
                        for query in insert_queries:
                            cursor.execute(query)
                        conn.commit()
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Found JSON file: C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/6041257.json
Connection established.
